In [ ]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets

W, H = 500, 500
canvas = Canvas(width=W, height=H, layout={'cursor': 'crosshair'})
# State variables for center and zoom
state = {'x': -0.5, 'y': 0, 'zoom': 1.0}

def draw():
    scale = 1.5 / state['zoom']
    x = np.linspace(state['x'] - scale, state['x'] + scale, W)
    y = np.linspace(state['y'] - scale, state['y'] + scale, H)
    X, Y = np.meshgrid(x, y)

    C = X + 1j * Y
    Z = np.zeros_like(C)
    fractal = np.zeros(C.shape, dtype=float)

    for i in range(100):
        mask = np.abs(Z) <= 2
        Z[mask] = Z[mask]**2 + C[mask]
        fractal[mask] += 1
    
    img = np.zeros((W, H, 3), dtype=np.uint8)
    img[:,:,0] = (fractal % 8) * 32     # Red "stripes"
    img[:,:,1] = (fractal % 16) * 16    # Green "depth"
    img[:,:,2] = (fractal % 32) * 8     # Blue "glow"
    with hold_canvas(canvas):
        canvas.put_image_data(img, 0, 0)

def on_click(x, y):
    # Map click (0 to W) to complex plane coordinates
    scale = 1.5 / state['zoom']
    state['x'] += (x / W - 0.5) * 2 * scale
    state['y'] += (y / H - 0.5) * 2 * scale
    state['zoom'] *= 2  # Zoom in by 2x on every click
    draw()

canvas.on_mouse_down(on_click)
display(canvas)
draw()